# Seyed Mohammad Amin Atyabi - 830402014 - HW 10

In [ ]:
import numpy
import pandas
from scipy import io as sio
from matplotlib import pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder


class NeuralNetwork:
    def __init__(self, x, y, test_x, test_y, learning_rate):
        self.input = x
        self.y = y
        self.test_x = test_x
        self.test_y = test_y
        self.output = numpy.zeros(self.y.shape)
        self.learning_rate = learning_rate
        self.layer1 = None
        self.weights1 = numpy.random.normal(0, 0.01, (self.input.shape[1], 2))
        self.weights2 = numpy.random.normal(0, 0.01, (2, 4))

    def activation_function(self, x):
        return (numpy.exp(2 * x) - 1) / (numpy.exp(2 * x) + 1)

    def activation_function_derivative(self, x):
        return (4 * numpy.exp(2 * x)) / (numpy.exp(2 * x) + 1) ** 2

    # def softmax(self, x):
    #     exps = numpy.exp(x - numpy.max(x, axis=1, keepdims=True))
    #     return exps / numpy.sum(exps, axis=1, keepdims=True)

    def feed_forward(self):
        self.layer1 = self.activation_function(numpy.dot(self.input, self.weights1))
        self.output = self.activation_function(numpy.dot(self.layer1, self.weights2))
        # self.output = self.softmax(numpy.dot(self.layer1, self.weights2))

    def back_propagate(self):
        d_weights2 = numpy.dot(self.layer1.T, (self.y - self.output))
        d_weights1 = numpy.dot(self.input.T, numpy.sum(numpy.dot(self.y - self.output, self.weights2.T), axis=1,
                                                       keepdims=True) * self.activation_function_derivative(
            self.layer1))

        self.weights1 += self.learning_rate * d_weights1
        self.weights2 += self.learning_rate * d_weights2

    def calculate_average_square_error(self):
        layer1 = self.activation_function(numpy.dot(self.test_x, self.weights1))
        output = self.activation_function(numpy.dot(layer1, self.weights2))
        return numpy.mean(numpy.square(self.test_y - output))


columns = [
    'AGE',
    'ON THYROXINE',
    'TSH MEASURED',
    'TSH',
    'T3',
    'TT4 MEASURED',
    'TT4',
    'SICKNESS'
]

data = pandas.DataFrame(sio.loadmat('Data/Thyroid.mat').get('Data'), columns=columns, dtype=float)

normalizer = Normalizer()
X = normalizer.fit_transform(data.drop('SICKNESS', axis=1).values)
y = data['SICKNESS'].values

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

nn = NeuralNetwork(X[0:80], y[0:80], X[80:], y[80:], 0.01)
error = []

for i in range(1000):
    nn.feed_forward()
    nn.back_propagate()
    error.append(nn.calculate_average_square_error())
    if error[-1] < 0.2:
        break

plt.plot(error, color='g')
plt.title('Average Square Error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.grid()
plt.show()